# Исследование надежности заемщиков

**Описание проекта:**

Заказчик — кредитный отдел банка. **Необходимо разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.** Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**План проекта:**

1. [Загрузка и описание данных](#Загрузка-и-описание-данных)
2. [Предобработка данных](#Предобработка-данных)
    1. [Обработка аномальных значений](#Обработка-аномальных-значений)
    2. [Удаление пропусков](#Удаление-пропусков)
    3. [Изменение типов данных](#Изменение-типов-данных)
    4. [Обработка дубликатов](#Обработка-дубликатов)
    5. [Категоризация данных](#Категоризация-данных)
    6. [Вывод](#Вывод)
3. [Исследовательский анализ данных](#Исследовательский-анализ-данных)
    1. [Зависимость между количеством детей и возвратом кредита в срок](#Зависимость-между-количеством-детей-и-возвратом-кредита-в-срок)
    2. [Зависимость между уровнем дохода и возвратом кредита в срок](#Зависимость-между-уровнем-дохода-и-возвратом-кредита-в-срок)
    3. [Зависимость между целью кредита и его возвратом в срок](#Зависимость-между-целью-кредита-и-его-возвратом-в-срок)
4. [Общий вывод](#Общий-вывод)

**Описание данных:**
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

## Загрузка и описание данных

In [1]:
# Импортируем библиотеку
import pandas as pd

In [2]:
# Загружаем датасет
data = pd.read_csv(r'C:\Users\ivank\practicum\datasets\data_bank_project.csv')

In [3]:
display(data.head())
data.info()
data.describe()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


В нашем распоряжении таблица, состоящая из 21 525 строк и 12 столбцов:

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

По таблице с информацией можно сказать, что в некоторых столбцах присутствуют пропуски, а также выбран неверный тип данных. 
Также мы можем видеть аномальные значения в столбце с количеством детей, например, 20 и -1, и отрицательные значения в столбце с трудовым стажем. Рассмотрим таблицу детальнее и обработаем недочеты в предобработке данных.

[Начало страницы](#Исследование-надежности-заемщиков)

## Предобработка данных

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Заменим отрицательные значения на положительные в данном столбце.

In [4]:
data['days_employed'] = data['days_employed'].abs()

Также обработаем значения в столбце с количеством детей. Рассмотрим, какие есть уникальные значения в данном столбце.

In [5]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

Из всех значений подозрения вызывают числа -1 и 20. Удалим столбцы с такими значениями.

In [6]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [7]:
# Проверяем уникальные значения
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков

In [8]:
# Считаем количество пропущенных строк
data.isna().sum()

children               0
days_employed       2162
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2162
purpose                0
dtype: int64

Пропуски присутствуют в столбцах с трудовым стажем и доходом клиентов банка. 

На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости сотрудник пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

Медианное значение является более объективным для количественных переменных, поскольку в данных могут встречаться значения, сильно выделяющиеся из остальных в большую или меньшую сторону, следовательно которые могут испортить результат, если рассчитывать средним арифметическим. С медианным значением такой проблемы не возникнет.

Пропуске в столбце со стажем можно заполнить по тому же принципу.

In [9]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()
    
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()    

In [10]:
# Проверяем, остались ли пропуски
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропусков в таблице не осталось, можно переходить к изменению типов данных.

### Изменение типов данных

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     21402 non-null  float64
 2   dob_years         21402 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      21402 non-null  float64
 11  purpose           21402 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Заменим вещественный тип данных в столбцах `total_income` и `days_employed` на целочисленный.

In [12]:
data['total_income'] = data['total_income'].astype(int)
data['days_employed'] = data['days_employed'].astype(int)

In [13]:
# Проверяем, изменился ли тип данных
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21402 non-null  int64 
 1   days_employed     21402 non-null  int32 
 2   dob_years         21402 non-null  int64 
 3   education         21402 non-null  object
 4   education_id      21402 non-null  int64 
 5   family_status     21402 non-null  object
 6   family_status_id  21402 non-null  int64 
 7   gender            21402 non-null  object
 8   income_type       21402 non-null  object
 9   debt              21402 non-null  int64 
 10  total_income      21402 non-null  int32 
 11  purpose           21402 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 2.0+ MB


Типы данных изменены, проверим, есть ли в таблице дубликаты.

### Обработка дубликатов

Ранее в таблице было видно, что в столбце с образованием уровень образования был написан разным регистром. Проверим, какие есть степени образования в датасете.

In [14]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

В датасете действительно присутствуют неявные дубликаты в данном столбце. Приведем все значения к нижнему регистру.

In [15]:
data['education'] = data['education'].str.lower()

In [16]:
# Проверим, удалились ли неявные дубликаты
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Проверим также другие данные типа `object` на неявные дубликаты.

In [17]:
# Проевряем столбец с семейным положением
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

В данном столбце все в порядке.

In [18]:
# Проверяем столбец с полом
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

В столбце с полом обнаружено неизвестное обозначение `XNA`. Просмотрим строки с данным полом.

In [19]:
data.query('gender == "XNA"')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


В датафрейме всего 1 строка с данным обозначением, можно ее удалить.

In [20]:
data = data.query('gender != "XNA"')

In [21]:
# Проверяем столбец с типом занятости
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

В данном столбце все в порядке.

In [22]:
# Проверяем столбец с целью получения кредита
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

В данном столбце много уникальных значений. Для упращения работы с ними категоризируем их в следующем пункте.

Теперь посмотрим, сколько стро-дубликатов есть в таблице.

In [23]:
data.duplicated().sum()

71

Всего в таблице 71 дубликат. Удалим данные строки.

In [24]:
data = data.drop_duplicates()

### Категоризация данных

Для дальнейшего анализа присвоим каждому клиенту категорию в зависимости от его уровня дохода. 
Создадим столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [25]:
# Создаем функцию для категоризации
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [26]:
# Добавляем столбец с категорией по уровню дохода
data['total_income_category'] = data['total_income'].apply(categorize_income)

Теперь снова рассмотрим уникальные значения цели кредита и создадим основные категории для данных целей.

In [27]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Судя по данным уникальным значениям люди пользуются кредитом со следующими целями:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Напишем функию, которая присвоит данные категории для каждого клиента.

In [28]:
# Создаем функию по подстрокам
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [29]:
# Добавляем столбец с категорией цели получения кредита
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Вывод

In [30]:
# Устанавливаем новые индексы после удаления строк
data = data.reset_index(drop=True)

In [31]:
display(data.head())
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21330 entries, 0 to 21329
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   children               21330 non-null  int64 
 1   days_employed          21330 non-null  int32 
 2   dob_years              21330 non-null  int64 
 3   education              21330 non-null  object
 4   education_id           21330 non-null  int64 
 5   family_status          21330 non-null  object
 6   family_status_id       21330 non-null  int64 
 7   gender                 21330 non-null  object
 8   income_type            21330 non-null  object
 9   debt                   21330 non-null  int64 
 10  total_income           21330 non-null  int32 
 11  purpose                21330 non-null  object
 12  total_income_category  21330 non-null  object
 13  purpose_category       21330 non-null  object
dtypes: int32(2), int64(5), object(7)
memory usage: 2.1+ MB


**По проведенной предобработке можно сделать следующий вывод:**

* Обработаны аномальные значения;
* Обработаны пропуски, заполнены пропущенные значения;
* Изменены типы данных;
* Обработаны явные и неявные дубликаты;
* Добавлены столбцы с категориями по уровню дохода и по целям получения кредита.

С данной таблицей можно приступать к анализу данных.

[Начало страницы](#Исследование-надежности-заемщиков)

## Исследовательский анализ данных

### Зависимость между количеством детей и возвратом кредита в срок

In [32]:
children_pivot = data.pivot_table(index='children', values='debt', 
                                  aggfunc=['count', 'sum', lambda x: '{:.2%}'.format(x.mean())])
children_pivot = children_pivot.rename(columns={'count': 'total', 'debt': '', 'sum': 'debt_amount', '<lambda>': 'ratio'})
children_pivot

,total,debt_amount,ratio
,,,
children,,,
0,14090,1063,7.54%
1,4808,444,9.23%
2,2052,194,9.45%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.00%


**Вывод:** Наименьший процент задолженности (7,54%) наблюдается у людей, не имеющих детей, в то время как наибольший процент у людей с четырьмя детьми (9,76%). Нельзя сказать, что у людей с пятью детьми не бывает просрочек по кредиту, поскольку данных о таких семьях слишком мало для анализа.

Кредитополучатели без детей являются немного более ответственными, чем клиенты у которых есть дети.

### Зависимость между уровнем дохода и возвратом кредита в срок

In [33]:
income_pivot = data.pivot_table(index='total_income_category', values='debt',
                                aggfunc=['count', 'sum', lambda x: '{:.2%}'.format(x.mean())])
income_pivot = income_pivot.rename(columns={'count': 'total', 'debt': '', 'sum': 'debt_amount', '<lambda>': 'ratio'})
income_pivot

,total,debt_amount,ratio
,,,
total_income_category,,,
A,25,2,8.00%
B,5013,354,7.06%
C,15921,1353,8.50%
D,349,21,6.02%
E,22,2,9.09%


**Вывод:** На первый взгляд, судя по доле просроченных кредитов, может показаться, что самыми исправными плательщиками являются заемщики группы D, а чаще просрочивают кредит заемщики группы E. На самом деле данных по этим группам, также включая группу A, недостаточно, чтобы делать подобные выводы, поэтому в данной выборке будет объективнее сделать вывод, основываясь на более многочисленных категориях. Таким образом, чаще выплачивают в срок кредит заемщики категории B, с уровнем дохода от 200 тыс. до 1 млн., доля просрочки кредитов составляет 7,06%, чаще же просрочивают кредит заемщики категории C, с уровнем дохода от 50 до 200 тыс., доля просрочки составляет 8,5%.

### Зависимость между целью кредита и его возвратом в срок

In [34]:
purpose_pivot = data.pivot_table(index='purpose_category', values='debt',
                                 aggfunc=['count', 'sum', lambda x: '{:.2%}'.format(x.mean())])
purpose_pivot = purpose_pivot.rename(columns={'count': 'total', 'debt': '', 'sum': 'debt_amount', '<lambda>': 'ratio'})
purpose_pivot

,total,debt_amount,ratio
,,,
purpose_category,,,
операции с автомобилем,4279,400,9.35%
операции с недвижимостью,10750,780,7.26%
получение образования,3988,369,9.25%
проведение свадьбы,2313,183,7.91%


**Вывод:** Наибольшая доля просрочки по кредиту приходится на людей, которые планируют получить образование (9,2%) и приобрести автомобиль (9,3%). Наиболее исправно выплачивают кредит люди, планирующие приобрести недвижимость, их доля просрочки состовляет 7,2%

Клиенты, бравшие кредит на автомобиль и образование, чаще нарушают сроки, в отличие от тех, кто получил кредит на недвижимость.

[Начало страницы](#Исследование-надежности-заемщиков)

## Общий вывод

**По итогам анализа приведенной выше таблицы можно сделать вывод о том, что:**

- Люди без детей чаще выплачивают кредит в срок. Чем больше детей, тем больше доля задолженности;
- Вдовы и вдовцы чаще выплачивают кредит в срок, чем неженатые и незамужние люди;
- Заемщики, желающие взять ипотечные кредиты и кредиты на образование, чаще выплачивают их вовремя;
- Люди с уровнем дохода от 200 тыс. до 1млн. реже просрочивают кредиты. Можно сделать вывод о том, что чем ниже уровень дохода заемщиков, тем реже они будут выплачивать кредит в срок.

[Начало страницы](#Исследование-надежности-заемщиков)